<html><center>
    <h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
    <br>
    by Célestin PHAM
    <br>
</center></html>

In [57]:
# import dependencies

import pandas as pd
import numpy as np

## Question 1 - Retrieve Neighborhoods from Wikipedia

In [58]:
import requests
from bs4 import BeautifulSoup

soup = BeautifulSoup(requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text, 'html.parser')


I will go through al 

In [65]:
zip_codes = []

# Look for all "td" tags in the wiki webpage
for tag in soup.find_all('td'):
    
    # Discard all "Not assigned" Borough, and not Postal Codes related "td" tags
    if not(tag.find("p")) or str(tag.i) == '<i>Not assigned</i>':
        continue
    
    to_parse = tag.p.span.text.replace(")", "").split("(")
    
    # Consider what is between brackets as the neighborhoods
    if len(to_parse)== 1:
        to_parse = to_parse[0].split("/")

    # Add to the zip codes list 
    zip_codes.append([
        tag.p.b.text.strip(),
        to_parse[0].strip(),
        to_parse[1].replace(" /", ",").strip()])
    
zip_codes = pd.DataFrame(data=zip_codes, columns=["PostalCode","Borough","Neighborhood"])
zip_codes.shape

(103, 3)